# Telco Classification Project

***

# Project Planning

Customers at the telecommunications company, Telco, are churning.  The goal of this project is to determine why 
customers are churning by developing machine learning classification models to predict customer churn based on the customer data available. The models will be created using Python, Pandas, Matplot, Seaborn, and Scikit-Learn libraries. 

### Business & Technical Goals

- Find drivers for customer churn at Telco. Why are customers churning?


- Construct a Machine Learning classification model that accurately predicts customer churn.

***

This dataset contains a total of 7,043 customers and 21 attributes, coming from personal characteristics, services signatures, and contract details. Out of the entries, 5,174 are active customers and 1,869 are churned, which demonstrates that the dataset is highly unbalanced. The target variable for this assessment is going to be the featureChurn.
The main goal is to develop a machine learning model capable to predict customer churn based on the customer’s data available. I will use mainly Python, Pandas, and Scikit-Learn libraries for this implementation. 


# Project Planning
***

#### Project Specs for Project Planning

- Describe the project and goals.  V

- Task out how you will work through the pipeline in as much detail as you need to keep on track.  

- Include a data dictionary to provide context for and explain your data.

- Clearly state your starting hypotheses (and add the testing of these to your task list).




#### EXAMPLE FROM IRIS NOTEBOOK:

- Create README.md with data dictionary, project and business goals, come up with initial hypotheses.
- Acquire data from the Codeup Database and create a function to automate this process. Save the function in an acquire.py file to import into the Final Report Notebook.
- Clean and prepare data for the first iteration through the pipeline, MVP preparation. Create a function to automate the process, store the function in a prepare.py module, and prepare data in Final Report Notebook by importing and using the funtion.
- Clearly define two hypotheses, set an alpha, run the statistical tests needed, reject or fail to reject the Null Hypothesis, and document findings and takeaways.
- Establish a baseline accuracy and document well.
- Train three different classification models.
- Evaluate models on train and validate datasets.
- Choose the model with that performs the best and evaluate that single model on the test dataset.
- Create csv file with the measurement id, the probability of the target values, and the model's prediction for each observation in my test dataset.
- Document conclusions, takeaways, and next steps in the Final Report Notebook.

# Executive Summary - Conclusions & Next Steps

- I found that all of the classification models I created, LogisticRegression, DecisionTree, RandomForest, and KNeighbors predicted the species of Iris equally well using the features sepal_width, sepal_length, petal_length, petal_width.
- I chose my DecisionTree model as my best model with a 90% accuracy rate for predicting my target value, species. This model outperformed my baseline score of 33% accuracy, so it has value.
- Some initial exploration and statistical testing revealed that engineering some new features like petal area or sepal area might help my models predict with even more accuracy, and with more time, I would like to test this hypothesis.